<a name='1.1'></a>
### 1.1 - Set up Kernel and Required Dependencies

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF")
secret_value_1 = user_secrets.get_secret("WB")

In [2]:
import wandb
import os
from huggingface_hub import login
wandb.login(key=secret_value_1)
login(token=secret_value_0)

wandb: Currently logged in as: reddyvarimallikarjunreddy (malli). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!pip3 install -q -U wandb
!pip3 install -q -U torchdata
!pip3 install -q -U torch
!pip3 install -q -U evaluate
!pip3 install -q -U rouge_score
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers
!pip3 install -q -U bitsandbytes


In [4]:
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

In [ ]:
!pip3 -q -U install trl[peft]
!pip3 -q -U install bitsandbytes loralib

Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook. 

In [83]:
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM,AutoModelForCausalLM,BitsAndBytesConfig, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a name='1.2'></a>
### 1.2 - Load Dataset and LLM

You are going to continue experimenting with the [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face dataset. It contains 10,000+ dialogues with the corresponding manually labeled summaries and topics. 

In [84]:
from datasets import Dataset
dataset_train = load_dataset("zqz979/meta-review", split="train")
dataset_eval = load_dataset("zqz979/meta-review", split="validation")
dataset_test = load_dataset("zqz979/meta-review", split="test")

def preprocess_dataset(dataset, input_column):
  def update_input(example):
    if example[input_column]:  # Check if empty
        if isinstance(example[input_column], str):  # Check if it's a string
            word_count = len(example["Output"].split())
            example[input_column] = f'summarize the following data in {word_count} words : {example[input_column]}'
        else:
            example[input_column] = "Data removed (not a string)"  # Placeholder for non-string data
            example["Output"] = "Output removed (input was not a string)"
    else:
        example[input_column] = "Data removed (empty)"  # Placeholder for empty data
        example["Output"] = "Output removed (input was empty)"
    return example

  return dataset.map(update_input) 


filtered_dataset_train =preprocess_dataset(dataset_train, input_column="Input")
filtered_dataset_eval = preprocess_dataset(dataset_eval, input_column="Input")
filtered_dataset_test = preprocess_dataset(dataset_test, input_column="Input")

In [88]:
model_name="allenai/led-base-16384" #led-large-16384
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    low_cpu_mem_usage=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=bnb_config).to("cuda:0")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


KeyError: 'final_logits_bias'

In [11]:
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [12]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(led))

trainable model parameters: 161844480
all model parameters: 161844480
percentage of trainable model parameters: 100.00%


In [13]:
max_input_length = 7168 # it is calculated
max_output_length = 512
batch_size = 1

In [15]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
rouge = load_metric("rouge")

In [23]:
print(led)

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0-5): 6 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
     

In [19]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
led.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(led)

In [39]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=128,
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout=0.15,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(led, lora_config)

In [80]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['Input'])):
        text = f" {example['Input'][i]}\n ### Answer: {example['Output'][i]}"
        output_texts.append(text)
    return output_texts

import transformers

from trl import SFTTrainer


trainer = SFTTrainer(
    model=led,
    train_dataset=filtered_dataset_train,
    compute_metrics=compute_metrics,
    eval_dataset=filtered_dataset_eval,
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        evaluation_strategy="steps",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        eval_steps=500,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        save_strategy="epoch",
    ),
     formatting_func=formatting_prompts_func,
  data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [81]:
trainer.train()

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/peft/peft_model.py", line 1279, in forward
    return self.base_model(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-